In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
import sklearn 
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
pd.options.display.float_format = "{:.2f}".format

In [46]:
dataset2=pd.read_csv("all_data.csv")

In [47]:
dataset2.head()

,Unnamed: 0,text,sentiment
0,0,﻿حقا\n,pos
1,1,صح جدا\n,pos
2,2,﻿اه و النعمة \n,pos
3,3,كلامك جميل ورائع\n,pos
4,4,﻿و خير الكلام ما قل و دل\n,pos


In [48]:
cols = ['sentiment','text']
positive = pd.read_csv('Arabic_tweets_positive.tsv',sep='\t', error_bad_lines = False ,header=None, names=cols)
positive.head()

,sentiment,text
0,pos,هه خلي فضولش ف جيبش 🌚
1,pos,موجود ماسافرت 😊
2,pos,: مسابقة #43 #متابعي_شامخ فقط# 🔘السحب على200💰ر...
3,pos,اللهم امين 🌺
4,pos,يعني اذا ما احبج احب منو؟ 💘


In [49]:
cols = ['sentiment','text']
negative = pd.read_csv('Arabic_tweets_negative.tsv',sep='\t', error_bad_lines = False ,header=None, names=cols)
negative.head()

,sentiment,text
0,neg,انمي Family Guy الموسم الحلقة السادسة مترجمة H...
1,neg,صدااع من الغباء لي ساعه افكر وش اقول بس غبائها...
2,neg,قتيلا وجريحا… حصيلة تفرقة المتظاهرين في كربلاء...
3,neg,اللهم من أصلح شأنهم وجمع كلمتهم ووحد صفهم ياقا...
4,neg,كوميديا.. الذهبية تذهب بالخطأ لصاحبة المركز ال...


In [50]:
dataset1 = pd.concat([positive, negative], axis=0)
dataset1.head()

,sentiment,text
0,pos,هه خلي فضولش ف جيبش 🌚
1,pos,موجود ماسافرت 😊
2,pos,: مسابقة #43 #متابعي_شامخ فقط# 🔘السحب على200💰ر...
3,pos,اللهم امين 🌺
4,pos,يعني اذا ما احبج احب منو؟ 💘


In [51]:
final_data = pd.concat([dataset1, dataset2], axis=0)
final_data.head()

,sentiment,text,Unnamed: 0
0,pos,هه خلي فضولش ف جيبش 🌚,NaN
1,pos,موجود ماسافرت 😊,NaN
2,pos,: مسابقة #43 #متابعي_شامخ فقط# 🔘السحب على200💰ر...,NaN
3,pos,اللهم امين 🌺,NaN
4,pos,يعني اذا ما احبج احب منو؟ 💘,NaN


In [52]:
final_data.sentiment.value_counts()

(304735, 3)

In [54]:
final_data.shape

(304735, 3)

In [55]:
final_data=final_data.drop_duplicates()
final_data.shape

(246361, 3)

In [56]:
final_data.sentiment.value_counts()

pos    137212
neg    109149
Name: sentiment, dtype: int64

In [57]:
finalpos= final_data[final_data.sentiment == "pos"]
finalpos.shape

(137212, 3)

In [58]:
finalneg= final_data[final_data.sentiment == "neg"]
finalneg.shape

(109149, 3)

In [59]:
finalpos = finalpos.sample(109149 , random_state=200)
finalpos.shape

(109149, 3)

In [60]:
final_data= pd.concat([finalpos, finalneg], axis=0)
final_data.head()

,sentiment,text,Unnamed: 0
34344,pos,على النقا والبياض وصافي النيه نعامل ناس صادقها...,32353.00
71631,pos,اليوم فيه 🎁 😍 السحب في أي وقت ✋ 🏼 واللي ما يحا...,69640.00
90361,pos,ياحبني لها الله يرزقنا دنيا وآخره يارب مميزه ❤...,88370.00
7785,pos,يارب تصير معجزه و افوز 🙂\n,5794.00
44212,pos,موضه الشباب اللحين يحط صورته ويكتب اصلا انا مو...,42221.00


In [61]:
final_data.sentiment.value_counts()

pos    109149
neg    109149
Name: sentiment, dtype: int64

In [62]:
final_data= final_data.drop('Unnamed: 0',axis=1)
final_data.head()

,sentiment,text
34344,pos,على النقا والبياض وصافي النيه نعامل ناس صادقها...
71631,pos,اليوم فيه 🎁 😍 السحب في أي وقت ✋ 🏼 واللي ما يحا...
90361,pos,ياحبني لها الله يرزقنا دنيا وآخره يارب مميزه ❤...
7785,pos,يارب تصير معجزه و افوز 🙂\n
44212,pos,موضه الشباب اللحين يحط صورته ويكتب اصلا انا مو...


In [63]:
final_data= final_data.drop_duplicates()
final_data.shape

(163498, 2)

In [65]:
print("tweets: ",len(final_data))

tweets:  163498


In [66]:
final_data.isnull().any(axis=0)

sentiment    False
text         False
dtype: bool

In [67]:
for letter in '#.][!XR':
    final_data['text'] = final_data['text'].astype(str).str.replace(letter,'')

In [68]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [69]:
def remove_non_arabic(text):
    return ' '.join(re.sub(u"[^\u0621-\u063A\u0640-\u0652 ]", " ", str(text),  flags=re.UNICODE).split())

In [70]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [71]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [72]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [73]:
def processPost(tweet): 

    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)
    
    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet= remove_punctuations(tweet)
    
    # normalize the tweet
    tweet= normalize_arabic(tweet)
    
    # remove repeated letters
    #tweet=remove_repeating_char(tweet)


    # remove emoji
    tweet=remove_emoji(tweet)
    
    return tweet

In [74]:
final_data["text"] = final_data['text'].apply(lambda x: processPost(x))

In [75]:
final_data["text"] = final_data['text'].apply(remove_non_arabic)

In [76]:
final_data.head()

,sentiment,text
34344,pos,علي النقا والبياض وصافي النيه نعامل ناس صادقها...
71631,pos,اليوم فيه السحب في اي وقت واللي ما يحالفه الحظ...
90361,pos,ياحبني لها الله يرزقنا دنيا واخره يارب مميزه ا...
7785,pos,يارب تصير معجزه و افوز
44212,pos,موضه الشباب اللحين يحط صورته ويكتب اصلا انا مو...


In [77]:
final_data.shape

(163498, 2)

In [78]:
final_data["text"] = final_data['text'].apply(lambda x:remove_repeating_char(x))

In [79]:
from nltk.corpus import stopwords
stopwords_list = stopwords.words('arabic')
print(len(stopwords_list))

754


In [80]:
listToStr = ' '.join([str(elem) for elem in stopwords_list])
listToStr

'إذ إذا إذما إذن أف أقل أكثر ألا إلا التي الذي الذين اللاتي اللائي اللتان اللتيا اللتين اللذان اللذين اللواتي إلى إليك إليكم إليكما إليكن أم أما أما إما أن إن إنا أنا أنت أنتم أنتما أنتن إنما إنه أنى أنى آه آها أو أولاء أولئك أوه آي أي أيها إي أين أين أينما إيه بخ بس بعد بعض بك بكم بكم بكما بكن بل بلى بما بماذا بمن بنا به بها بهم بهما بهن بي بين بيد تلك تلكم تلكما ته تي تين تينك ثم ثمة حاشا حبذا حتى حيث حيثما حين خلا دون ذا ذات ذاك ذان ذانك ذلك ذلكم ذلكما ذلكن ذه ذو ذوا ذواتا ذواتي ذي ذين ذينك ريث سوف سوى شتان عدا عسى عل على عليك عليه عما عن عند غير فإذا فإن فلا فمن في فيم فيما فيه فيها قد كأن كأنما كأي كأين كذا كذلك كل كلا كلاهما كلتا كلما كليكما كليهما كم كم كما كي كيت كيف كيفما لا لاسيما لدى لست لستم لستما لستن لسن لسنا لعل لك لكم لكما لكن لكنما لكي لكيلا لم لما لن لنا له لها لهم لهما لهن لو لولا لوما لي لئن ليت ليس ليسا ليست ليستا ليسوا ما ماذا متى مذ مع مما ممن من منه منها منذ مه مهما نحن نحو نعم ها هاتان هاته هاتي هاتين هاك هاهنا هذا هذان هذه هذي هذين هكذا هل هلا هم هما هن هنا هن

In [81]:
final_data["text"]=final_data["text"].apply(lambda x: [item for item in x if item not in stopwords_list])

In [83]:
all_words = [word for tokens in final_data["text"] for word in tokens]
sentence_lengths = [len(tokens) for tokens in final_data["text"]]

VOCAB = sorted(list(set(all_words)))

print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max sentence length is %s" % max(sentence_lengths))

1510896 words total, with a vocabulary size of 9
Max sentence length is 1220


In [84]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential

In [85]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import text,sequence

max_fatures = 10000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(final_data['text'].values)
X = tokenizer.texts_to_sequences(final_data['text'].values)
X = pad_sequences(X)

In [86]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1220, 128)         1280000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1220, 128)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 1,535,194
Trainable params: 1,535,194
Non-trainable params: 0
_________________________________________________________________
None


In [87]:
Y = pd.get_dummies(final_data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(130798, 1220) (130798, 2)
(32700, 1220) (32700, 2)


In [88]:
batch_size = 16
model.fit(X_train, Y_train, epochs = 3, batch_size=batch_size, verbose = 1)

Epoch 1/3
8175/8175 [==============================] - 13908s 2s/step - loss: 0.6925 - accuracy: 0.5178
Epoch 2/3
8175/8175 [==============================] - 13783s 2s/step - loss: 0.6912 - accuracy: 0.5261
Epoch 3/3
8175/8175 [==============================] - 13811s 2s/step - loss: 0.6909 - accuracy: 0.5265


In [89]:
import keras.backend as K

def f1_score(precision, recall):
    ''' Function to calculate f1 score '''
    
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [90]:
# Evaluate model on the test set
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
precision,recall= model.evaluate(X_test, Y_test, verbose=1, batch_size = batch_size)
# Print metrics
print('')
print('Precision : {:.4f}'.format(precision))
print('Recall    : {:.4f}'.format(recall))
print('F1 Score  : {:.4f}'.format(f1_score(precision, recall)))


1950/1950 [==============================] - 451s 231ms/step - loss: 0.6906 - accuracy: 0.5229

Precision : 0.6906
Recall    : 0.5229
F1 Score  : 0.5952
